In [6]:
import numpy as np
from scipy.ndimage import imread
from PIL import Image
import cv2
import matplotlib.pyplot as plt

In [7]:
def get_simMatrix(X):
    centroid = np.mean(X, axis=0)
    x_normalized = X - centroid
    s = np.sqrt(2)/np.mean(np.sqrt(np.sum(x_normalized**2, axis=1)))
    x_normalized = x_normalized * s
    x_normalized = make_homog(x_normalized)
    sim_matrix= np.array([[s, 0, -s * centroid[0]],[0, s, -s * centroid[1]],[0, 0, 1]])
    return (sim_matrix, x_normalized)

def make_homog(x):
    return np.append(x, np.ones((len(x), 1)), axis=1)


def dlt(x, x_tick):
    '''
    Perform DLT algorithm
    '''
    # Dimension check
    if x.shape != x_tick.shape:
        print("Input arrays must have same dimensions")
        return None

    # Construct matrix A in equation A.H = 0
    A = None
    zeros = np.zeros(3)
    for i in range(len(x)):
        # First row of A_i [0  -w'x  y'x]
        A_i_0 = zeros
        A_i_0 = np.append(A_i_0, - x_tick[i][2]*x[i])
        A_i_0 = np.append(A_i_0,   x_tick[i][1]*x[i])
        A_i_0 = A_i_0.reshape(1, len(A_i_0))
        # Second row of A_i [w'x  0  -x'x]
        A_i_1 = x_tick[i][2]*x[i]
        A_i_1 = np.append(A_i_1,   zeros)
        A_i_1 = np.append(A_i_1, - x_tick[i][0]*x[i])
        A_i_1 = A_i_1.reshape(1, len(A_i_1))
        # Append to A
        if A is None:
            A = A_i_0
        else:
            A = np.append(A, A_i_0, axis=0)
        A = np.append(A, A_i_1, axis=0)

    # SVD decomposition, H is the eigenvector corresponding to the
    # smallest singular value
    u, s, v = np.linalg.svd(A, full_matrices=False)
    H = v[np.argmin(s)].T.reshape(3, 3)
    # Divide by H[2][2] for correct scaling? Not sure why this worked
    H = H/H[2][2]
    return H

def drawlines(img1,img2,lines,pts1,pts2):
    ''' img1 - image on which we draw the epilines for the points in img2
        lines - corresponding epilines '''
#     print img1.shape
    r,c = img1.shape
    img1 = cv2.cvtColor(img1,cv2.COLOR_GRAY2BGR)
#     print img1
    img2 = cv2.cvtColor(img2,cv2.COLOR_GRAY2BGR)
    for r,pt1,pt2 in zip(lines,pts1,pts2):
#         color = tuple(np.random.randint(0,255,3).tolist())
        color = tuple((255,0,0))
        x0,y0 = map(int, [0, -r[2]/r[1] ])
        x1,y1 = map(int, [c, -(r[2]+r[0]*c)/r[1] ])
#         y0 = 0
#         y1 = 100
        cv2.line(img1, (x0,y0), (x1,y1), color,1)
#         print x0,y0,x1,y1,img1
#         cv2.circle(img1,tuple(pt1),5,color,-1)
#         cv2.circle(img2,tuple(pt2),5,color,-1)
#     print img1.shape
    return img1,img2


In [8]:
def estimateFundamentalMatrix(inlierPts1, inlierPts2):
    #Slide 53 04-MultipleView
    
    #Normalization through a similarity Matrix
    simMatrix1, inliersPts1_Normalized = get_simMatrix(inlierPts1)
    simMatrix2, inliersPts2_Normalized = get_simMatrix(inlierPts2)
    
    #Linear Solution
    assert inliersPts1_Normalized.shape == inliersPts2_Normalized.shape
    number_elements = len(inliersPts1_Normalized)
    
    inliersPts1_Normalized = inliersPts1_Normalized.reshape(number_elements,3)
    inliersPts2_Normalized = inliersPts2_Normalized.reshape(number_elements,3)
        
    # Construct matrix A in equation A.H = 0
    
    H = dlt(inliersPts1_Normalized,inliersPts1_Normalized)
#     zero = np.zeros(3)
#     A1= zero
#     A1= np.append(A1, - inliersPts2_Normalized[0][2]*inliersPts1_Normalized[0])
#     A1= np.append(A1,   inliersPts2_Normalized[0][1]*inliersPts1_Normalized[0])
#     A1= A1.reshape(1, len(A1))
#     A2 = inliersPts2_Normalized[0][2]*inliersPts1_Normalized[0]
#     A2 = np.append(A2,   zero)
#     A2 = np.append(A2, - inliersPts2_Normalized[0][0]*inliersPts1_Normalized[0])
#     A2 = A2.reshape(1, len(A2))    
    
#     A = A1
#     A = np.append(A, A2, axis=0)
    
#     for i in range(1,number_elements):
        
#         A1= zero
#         A1= np.append(A1, - inliersPts2_Normalized[i][2]*inliersPts1_Normalized[i])
#         A1= np.append(A1,   inliersPts2_Normalized[i][1]*inliersPts1_Normalized[i])
#         A1= A1.reshape(1, len(A1))
           
        
#         A2 = inliersPts2_Normalized[i][2]*inliersPts1_Normalized[i]
#         A2 = np.append(A2,   zero)
#         A2 = np.append(A2, - inliersPts2_Normalized[i][0]*inliersPts1_Normalized[i])
#         A2 = A2.reshape(1, len(A2))
        
#         A = np.append(A, A1, axis=0)
#         A = np.append(A, A2, axis=0)
    
#     A = np.array(A)
    
#     H = cv2.findHomography(inlierPts1_cv.reshape(-1,1,2).astype(float),inlierPts2_cv.reshape(-1,1,2).astype(float))
#     print H[0]
#     U, S, V = np.linalg.svd(H, full_matrices=True)
#     print "S ", S , " V " , V.shape
#     hat_F = V[:,np.argmin(S)].reshape(3, 3)
#     print "Arg min",np.argmin(S)
#     hat_F= hat_F/hat_F[2][2]
    
#     print "Linear solution\n", hat_F
    
#     Singularity Enforcement
    
    print H
    u,s,v = np.linalg.svd(H)
    print s,s.shape,np.diag([s[0],s[-1],0])
    H_norm = np.dot(u,np.dot(np.diag([s[0],s[-1],0]),v.T))
    print "H norm" , H_norm
#     for i in range(3,number_elements*2):
#         H = dlt(inliersPts1_Normalized[:i],inliersPts1_Normalized[:i])
# #         U, S, V = np.linalg.svd(H, full_matrices=True)
# #         print "V " , V.shape
# #         hat_F_tick = Vnp.argmin(S)]
# #         print hat_F_tick
# #         hat_F_tick = hat_F_tick/hat_F_tick[2][2]
# #         print "F' " , hat_F_tick
        
#         if abs(np.linalg.det(H)) <= 0.01:
#             print np.linalg.det(H)
#             print "Number of Lines: ", i
#             break;
#     print "Singularity Reinforcement\n", H
    
    
    #Denormalization
    F = np.dot(simMatrix2.T,np.dot(H_norm,simMatrix1))
    F = F/F[-1,-1]
#     F = np.dot(simMatrix2,np.dot(hat_F,simMatrix1))
    
#     print "Denormalization\n", F
#     F = F/F[-1,-1]
    
#     return F

    return F

In [11]:
path = '/home/jose/Dropbox/MAS/Semester_2/Robot Perception/HW/HW 08/FMatriximages/FMatriximages/'

img1 = Image.open(path+str(1)+'/IMG_0350-stereo.jpg')

homography = open(path+str(7)+'/FMatrixCorrPointsStereo.txt').readlines()

#Get number of lines of correspondences points points
left_start = homography.index(">> inlierPtsL.Location\n")
right_start = homography.index(">> inlierPtsR.Location\n")

#Separate textfile
L_correspondences = homography[left_start+4:right_start-1]
R_correspondences = homography[right_start+4:len(homography)]

#Replace Spaces by ;
L_correspondences = ([i.replace('        ',';').split(';') for i in L_correspondences])
R_correspondences = ([i.replace('        ',';').split(';') for i in R_correspondences])

#pop first space
for i in L_correspondences:
    i.pop(i.index(''))

#pop first space second points
for i in R_correspondences:
    i.pop(i.index(''))

#Delete chomp
for i in L_correspondences:
    i[1].rstrip()

#Delete chomp second points
for i in R_correspondences:
    i[1].rstrip()

#str to float
for i in L_correspondences:
    i[0] = float(i[0])
    i[1] = float(i[1])

#str to float second points
for i in R_correspondences:
    i[0] = float(i[0])
    i[1] = float(i[1])

#Np Arrays
inlierPts1 = np.asarray((L_correspondences))
inlierPts2 = np.asarray((R_correspondences))

assert inlierPts1.shape == inlierPts2.shape
print "Done"



Done


In [12]:
fNorm8Point = estimateFundamentalMatrix(inlierPts1, inlierPts2)
print "F " , fNorm8Point
F = fNorm8Point

[[  1.00000000e+00  -1.92296269e-16  -8.03101904e-17]
 [  7.21111007e-17   1.00000000e+00  -1.08166651e-16]
 [  9.61481343e-17   1.20185168e-16   1.00000000e+00]]
[ 1.  1.  1.] (3,) [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  0.]]
H norm [[  7.69936700e-18  -5.84261267e-17   1.04745320e-16]
 [ -1.29530121e-01   9.82930583e-01  -1.30649980e-01]
 [  1.70694174e-02  -1.29530121e-01  -9.91428556e-01]]
F  [[  1.45876849e-22  -1.10697662e-21   5.46612017e-19]
 [ -2.45415578e-06   1.86231955e-05  -4.58538851e-03]
 [  7.61955723e-04  -5.78204957e-03   1.00000000e+00]]


In [24]:
inlierPts1_cv =  np.int32(inlierPts1)
inlierPts2_cv =  np.int32(inlierPts2)

img1 = cv2.imread('/home/jose/Dropbox/MAS/Semester_2/Robot Perception/HW/HW 08/FMatriximages/FMatriximages/7/picture-24.jpg',0)
print img1.shape
# img2 = cv2.imread('/home/anand/Studies/SEM2/Robot_Perception/RobotPerception/HW 08/FMatriximages/FMatriximages/6/stereo2.jpg',0)
F_cv, mask = cv2.findFundamentalMat(inlierPts1_cv.reshape(-1,1,2).astype(float),inlierPts2_cv.reshape(-1,1,2).astype(float),cv2.FM_8POINT)
print F_cv

lines1 = cv2.computeCorrespondEpilines(inlierPts2_cv.reshape((-1,1,2)).astype('float32'), 2,F.T)
# print lines1
lines1 = lines1.reshape((-1,3))
# print lines1.shape
print cv2.polylines(img1,np.int32([lines1[:,0:2]]),False,(0,255,255))
# print cv2.line(img1,np.int32(lines1[0,0:2]))
# print lines1 ,lines1.shape
img5,img6 = drawlines(img1,img1,lines1,inlierPts1_cv,inlierPts2_cv)
# print img5
# Find epilines corresponding to points in left image (first image) and
# drawing its lines on right image
# lines2 = cv2.computeCorrespondEpilines(inlierPts1_cv.reshape((-1,1,2)).astype('float32'), 1,F)
# lines2 = lines2.reshape(-1,3)
# img3,img4 = drawlines(img2,img1,lines2,inlierPts2_cv,inlierPts1_cv)
# print img3
# print F - F_cv


fig1 = plt.gcf()
plt.imshow(img5)
# plt.subplot(122),plt.imshow(img3)
plt.show()
fig1.savefig('lines.jpg')
# cv2.imshow(img1)

(636, 1920)
[[  4.37070032e-08  -1.30777180e-04   6.77950117e-02]
 [  1.31947813e-04   6.81269331e-06   1.27319352e+00]
 [ -6.79503987e-02  -1.27817883e+00   1.00000000e+00]]
None
